In [4]:
import pandas as pd

data_noise = pd.read_csv("../Data/hourly_noisedata_2022.csv")
data_noise.head(20)

,month,day,hour,description,lamax,laeq
0,1,1,0,MP 03: Naamsestraat 62 Taste,60.322528,57.126833
1,1,1,0,MP 05: Calvariekapel KU Leuven,53.230972,49.987639
2,1,1,0,MP 06: Parkstraat 2 La Filosovia,53.666056,50.752000
3,1,1,0,MP 07: Naamsestraat 81,50.056861,47.440222
4,1,1,1,MP 03: Naamsestraat 62 Taste,53.033583,50.853806
5,1,1,1,MP 05: Calvariekapel KU Leuven,53.599639,50.578806
6,1,1,1,MP 06: Parkstraat 2 La Filosovia,59.880694,56.942167
7,1,1,1,MP 07: Naamsestraat 81,50.097278,47.878333
8,1,1,2,MP 03: Naamsestraat 62 Taste,52.173702,50.049903
9,1,1,2,MP 05: Calvariekapel KU Leuven,51.078083,47.974361


In [6]:
print(len(data_noise))
print(len(data_noise)/4/24/365) #see if there is one datapoint per hour for every location over the complete year

55469
1.5830194063926941


In [9]:
#data Naamsestraat
data_Namen = data_noise[data_noise["description"]=="MP 03: Naamsestraat 62 Taste"]

print(len(data_Namen))
print(len(data_Namen)/24)
#so this dataset is complete

8759
364.9583333333333


In [12]:
import plotly.graph_objects as go


# Creating the scatter plot using go.Scatter
fig = go.Figure(data=go.Scatter(
    #x=data_Namen['time'],
    y=data_Namen['laeq'],
    mode='lines'
))

# Customizing the plot layout
fig.update_layout(
    title='Scatter Plot of LAEQ over Time',
    xaxis=dict(title='Time (hours)'),
    yaxis=dict(title='LAEQ')
)

# Displaying the plot
fig.show()


In [120]:
#use of Fourrier transformation, but possible artefacts as it is not perfectly periodical and due to peaks in the time series

#use fast Fourier transformations as the data is discrete

import numpy as np
from scipy.fft import fft, fftfreq
import plotly.graph_objects as go

# Number of sample points
N = len(data_Namen)
# Sample spacing
T = 3600  # in seconds (1 hour)
x = np.linspace(0.0, N * T, N, endpoint=False)
y = data_Namen["laeq"].to_numpy()

# Compute the FFT
yf = fft(y)
xf = fftfreq(N, T)[:N//2]

# Create the Plotly scatter plot
fig = go.Figure(data=go.Scatter(x=3600*xf, y=2.0/N * np.abs(yf[0:N//2]), mode='lines'))

# Customize the plot layout
fig.update_layout(
    title="Frequency Spectrum",
    xaxis_title="Frequency (1/hour)",
    yaxis_title="Amplitude",
    xaxis_type="log",
    yaxis_type="log",
    showlegend=False
)

# Add vertical lines
x_lines = [1 / 24, 1 / 720, 1 / 2160, 1 / 2,      1/(7*24), 2/(7*24),    1/12,           1/(24*(365/2)),  1/(24*365)]
labels = ["day",   "month", "season", "day-night","week",   "half-week", "12 hours",      "half year",     "year"]
for x, label in zip(x_lines, labels):
    fig.add_shape(type="line",
                  x0=x,
                  y0=fig.data[0].y.min(),
                  x1=x,
                  y1=fig.data[0].y.max(),
                  line=dict(color="red", width=1, dash="dash")
                  )
    fig.add_annotation(
        x=np.log10(x),
        y=np.log10(fig.data[0].y.max()),  # Adjust the y position
        text=label,
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-50  # Adjust the y offset
    )
# Display the plot
fig.show()

#peak 52days
#peak 33.6 hours (0.0297)
#peak 21 hours (0.0476)


In [31]:
from scipy.fft import ifft

# Selected frequencies
selected_frequencies = [1 / 24, 1 / 720, 1 / 2160, 1 / 2, 1 / (7 * 24), 2 / (7 * 24), 1 / 12, 1 / (24 * (365 / 2)), 1 / (24 * 30)]
selected_frequencies = [freq / 3600 for freq in selected_frequencies]  # Convert to Hz

# Find the indices corresponding to the selected frequencies
selected_indices = [np.abs(xf - freq).argmin() for freq in selected_frequencies]

# Retrieve the selected peaks from the FFT result
selected_peaks = yf[selected_indices]

# Perform the inverse FFT (iFFT) to retransform to the time domain
retransformed_signal = ifft(selected_peaks)

x = np.linspace(0.0, N * T, N, endpoint=False)

# Create the Plotly line plot for the retransformed signal
fig = go.Figure(data=go.Scatter(x=x, y=np.real(retransformed_signal), mode='lines'))

# Customize the plot layout
fig.update_layout(
    title="Retransformed Signal",
    xaxis_title="Time (s)",
    yaxis_title="Amplitude",
    showlegend=False
)

# Display the plot
fig.show()

In [34]:
#do all the same job as above, but with standardized data
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

laeq = data_Namen["laeq"]

pipeline = Pipeline([
    ('scaler', StandardScaler())
])

laeq_standardized = pipeline.fit_transform(laeq.values.reshape(-1, 1))

In [40]:
print(laeq_standardized)

#make it a 1D array instead of 2D
laeq_standardized = laeq_standardized.flatten()
print(laeq_standardized)

[[ 1.0716033 ]
 [-0.56307828]
 [-0.77256643]
 ...
 [-0.67493607]
 [-0.51693956]
 [ 0.95021671]]
[ 1.0716033  -0.56307828 -0.77256643 ... -0.67493607 -0.51693956
  0.95021671]


In [41]:
#plot with standardized values

# Creating the scatter plot using go.Scatter
fig = go.Figure(data=go.Scatter(
    #x=data_Namen['time'],
    y=laeq_standardized,
    mode='lines'
))

# Customizing the plot layout
fig.update_layout(
    title='Scatter Plot of LAEQ over Time',
    xaxis=dict(title='Time (hours)'),
    yaxis=dict(title='LAEQ')
)

# Displaying the plot
fig.show()

In [74]:
# Number of sample points
N = len(laeq_standardized)
# Sample spacing
T = 3600  # in seconds (1 hour)
x = np.linspace(0.0, N * T, N, endpoint=False)
y = laeq_standardized

# Compute the FFT
yf = fft(y)
xf = fftfreq(N, T)[:N//2]

# Create the Plotly scatter plot
fig = go.Figure(data=go.Scatter(x=3600*xf, y=2.0/N * np.abs(yf[0:N//2]), mode='lines'))

# Customize the plot layout
fig.update_layout(
    title="Frequency Spectrum",
    xaxis_title="Frequency (1/hour)",
    yaxis_title="Amplitude",
    xaxis_type="log",
    yaxis_type="log",
    showlegend=False
)

# Add vertical lines
x_lines = [1 / 24, 1 / 720, 1 / 2160, 1 / 2, 1/(7*24), 2/(7*24), 1/12, 1/(24*(365/2)), 1/(24*365)]
labels = ["day","month","season","2 hours","week","half-week","12 hours","half year","year"]
for x, label in zip(x_lines, labels):
    fig.add_shape(type="line",
                  x0=x,
                  y0=fig.data[0].y.min(),
                  x1=x,
                  y1=fig.data[0].y.max(),
                  line=dict(color="red", width=1, dash="dash")
                  )
    fig.add_annotation(
        x=np.log10(x),
        y=np.log10(fig.data[0].y.max()),  # Adjust the y position
        text=label,
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-50  # Adjust the y offset
    )
# Display the plot
fig.show()

#peak 52days
#peak 33.6 hours (0.0297)
#peak 21 hours (0.0476)

In [77]:
# Selected frequencies
selected_frequencies = [0, 1 / 24, 1 / 720, 1 / 2160, 1 / 2, 1 / (7 * 24), 2 / (7 * 24), 1 / 12, 1 / (24 * (365 / 2)), 1/(24*365)]
#0 frequency for the "intercept"
selected_frequencies = [freq / 3600 for freq in selected_frequencies]  # Convert to Hz

# Find the indices corresponding to the selected frequencies
selected_indices = [np.abs(xf - freq).argmin() for freq in selected_frequencies]

# Retrieve the selected peaks from the FFT result
selected_peaks = yf[selected_indices]

# Perform the inverse FFT (iFFT) to retransform to the time domain
retransformed_signal = ifft(selected_peaks)

x = np.linspace(0.0, N * T, N, endpoint=False)

# Create the Plotly line plot for the retransformed signal
fig = go.Figure(data=go.Scatter(x=x, y=np.real(retransformed_signal), mode='lines'))

# Customize the plot layout
fig.update_layout(
    title="Retransformed Signal",
    xaxis_title="Time (s)",
    yaxis_title="Amplitude",
    showlegend=False
)

# Display the plot
fig.show()

In [47]:
print(xf)

print(selected_indices)

print("selected peak day")
print(yf[365])
print(yf[364])
print(yf[366])

[0.00000000e+00 3.17134122e-08 6.34268245e-08 ... 1.38777892e-04
 1.38809605e-04 1.38841319e-04]
[0, 365, 12, 4, 4378, 52, 104, 730, 2, 12]
selected peak day
(-3465.437622970978+2786.4891624597262j)
(342.7127866555655+88.278692707462j)
(277.6186967568573+215.15750500724255j)


In [92]:
#reconstruct by hand

amplitude = selected_peaks[2]

print(amplitude)
frequency = selected_frequencies[2]

print(frequency)

print(len(data_Namen))

t = np.linspace(0, len(data_Namen)-1,num=len(data_Namen))
print(t)
reconstructed_signal = np.real(amplitude * np.exp(1j * 2*np.pi* frequency * (t*3600)))


#plot with standardized values

# Creating the scatter plot using go.Scatter
fig = go.Figure(data=go.Scatter(
    #x=data_Namen['time'],
    x = t,
    y=reconstructed_signal,
    mode='lines'
))

# Customizing the plot layout
fig.update_layout(
    title='Scatter Plot of LAEQ over Time',
    xaxis=dict(title='Time (hours)'),
    yaxis=dict(title='LAEQ')
)

# Displaying the plot
fig.show()

(-78.00484210982455+245.82495260141826j)
3.858024691358025e-07
8759
[0.000e+00 1.000e+00 2.000e+00 ... 8.756e+03 8.757e+03 8.758e+03]


In [88]:
#reconstruct by hand

# Selected frequencies
selected_frequencies = [0, 1 / 24, 1 / 720, 1 / 2160, 1 / 2, 1 / (7 * 24), 2 / (7 * 24), 1 / 12, 1 / (24 * (365 / 2)), 1/(24*365)]
#0 frequency for the "intercept"
selected_frequencies = [freq / 3600 for freq in selected_frequencies]  # Convert to Hz

# Find the indices corresponding to the selected frequencies
selected_indices = [np.abs(xf - freq).argmin() for freq in selected_frequencies]

# Retrieve the selected peaks from the FFT result
selected_peaks = yf[selected_indices]

amplitude = selected_peaks

frequency = selected_frequencies

t = np.linspace(0, len(data_Namen)-1,num=len(data_Namen))

reconstructed_signal = np.zeros(len(t))#, dtype=np.complex128)  # Initialize the reconstructed signal

signals_array = []
amplitude_signals = np.zeros(len(frequency))

for i in range(len(frequency)):
    extra_signal = np.real(amplitude[i] * np.exp(1j * 2 * np.pi * frequency[i] * t))
    amplitude_extra_signal = max(extra_signal)
    signals_array.append(extra_signal)
    amplitude_signals[i] = amplitude_extra_signal

amplitude_day = 2
print(amplitude_signals)
max_amplitudes = max(amplitude_signals)
print("max amplitude")
print(max_amplitudes)
amplitude_signals = amplitude_signals*amplitude_day/max_amplitudes #rescale everything

print(signals_array[0])

for i in range(len(signals_array)):
    
    reconstructed_signal += signals_array[i]*amplitude_day/max_amplitudes

#plot with standardized values

# Creating the scatter plot using go.Scatter
fig = go.Figure(data=go.Scatter(
    #x=data_Namen['time'],
    x = t,
    y=reconstructed_signal,
    mode='lines'
))

# Customizing the plot layout
fig.update_layout(
    title='Scatter Plot of LAEQ over Time',
    xaxis=dict(title='Time (hours)'),
    yaxis=dict(title='LAEQ')
)

# Displaying the plot
fig.show()

[ 7.91899879e-12 -3.46543762e+03 -7.80048421e+01 -7.22061642e+01
  3.53818643e+01 -7.40658592e+02 -2.67964894e+02 -2.11111814e+02
 -8.90876128e+02  8.97664411e+01]
max amplitude
89.76644108270504
[7.91899879e-12 7.91899879e-12 7.91899879e-12 ... 7.91899879e-12
 7.91899879e-12 7.91899879e-12]


In [104]:
#reconstruct by hand

amplitude = selected_peaks[1]
frequency = selected_frequencies[1]
t = np.linspace(0, len(data_Namen)-1,num=len(data_Namen))
reconstructed_signal = np.real(amplitude * np.exp(1j * 2*np.pi* frequency * (t*3600)))

##############################################################################################
amplitude = selected_peaks[9]
frequency = selected_frequencies[9]
t = np.linspace(0, len(data_Namen)-1,num=len(data_Namen))
reconstructed_signal += np.real(amplitude * np.exp(1j * 2*np.pi* frequency * (t*3600)))

##############################################################################################
amplitude = selected_peaks[2]
frequency = selected_frequencies[2]
t = np.linspace(0, len(data_Namen)-1,num=len(data_Namen))
reconstructed_signal += np.real(amplitude * np.exp(1j * 2*np.pi* frequency * (t*3600)))

##############################################################################################
amplitude = selected_peaks[3]
frequency = selected_frequencies[3]
t = np.linspace(0, len(data_Namen)-1,num=len(data_Namen))
reconstructed_signal += np.real(amplitude * np.exp(1j * 2*np.pi* frequency * (t*3600)))

##############################################################################################
amplitude = selected_peaks[4]
frequency = selected_frequencies[4]
t = np.linspace(0, len(data_Namen)-1,num=len(data_Namen))
reconstructed_signal += np.real(amplitude * np.exp(1j * 2*np.pi* frequency * (t*3600)))

##############################################################################################
amplitude = selected_peaks[5]
frequency = selected_frequencies[5]
t = np.linspace(0, len(data_Namen)-1,num=len(data_Namen))
reconstructed_signal += np.real(amplitude * np.exp(1j * 2*np.pi* frequency * (t*3600)))

##############################################################################################
amplitude = selected_peaks[6]
frequency = selected_frequencies[6]
t = np.linspace(0, len(data_Namen)-1,num=len(data_Namen))
reconstructed_signal += np.real(amplitude * np.exp(1j * 2*np.pi* frequency * (t*3600)))

##############################################################################################
amplitude = selected_peaks[7]
frequency = selected_frequencies[7]
t = np.linspace(0, len(data_Namen)-1,num=len(data_Namen))
reconstructed_signal += np.real(amplitude * np.exp(1j * 2*np.pi* frequency * (t*3600)))

##############################################################################################
amplitude = selected_peaks[8]
frequency = selected_frequencies[8]
t = np.linspace(0, len(data_Namen)-1,num=len(data_Namen))
reconstructed_signal += np.real(amplitude * np.exp(1j * 2*np.pi* frequency * (t*3600)))

reconstructed_signal = reconstructed_signal*2/max(reconstructed_signal)

#plot with standardized values

# Creating the scatter plot using go.Scatter
fig = go.Figure(data=go.Scatter(
    #x=data_Namen['time'],
    x = t,
    y=reconstructed_signal,
    mode='lines'
))

# Customizing the plot layout
fig.update_layout(
    title='Scatter Plot of LAEQ over Time',
    xaxis=dict(title='Time (hours)'),
    yaxis=dict(title='LAEQ')
)

# Displaying the plot
fig.show()

In [115]:
# Creating the scatter plot for reconstructed_signal
trace1 = go.Scatter(
    x=t,
    y=reconstructed_signal,
    mode='lines',
    name='Reconstructed Signal',
    line=dict(color='red')
)

# Creating the scatter plot for laeq_standardized
trace2 = go.Scatter(
    y=laeq_standardized,
    mode='lines',
    name='LAEQ Standardized',
    opacity=0.5,
    line=dict(color='blue')
)

# Create the combined plot
fig = go.Figure(data=[trace1, trace2])

# Customizing the plot layout
fig.update_layout(
    title='Scatter Plot of LAEQ over Time',
    xaxis=dict(title='Time (hours)'),
    yaxis=dict(title='LAEQ')
)

# Display the plot
fig.show()

In [118]:
#reconstruct by hand

t = np.linspace(0, len(data_Namen)-1,num=len(data_Namen))
reconstructed_signal = np.zeros(len(t))

for i in range(len(selected_peaks)):
    amplitude = selected_peaks[i]
    frequency = selected_frequencies[i]
    reconstructed_signal += np.real(amplitude * np.exp(1j * 2*np.pi* frequency * (t*3600)))


reconstructed_signal = reconstructed_signal*2/max(reconstructed_signal)

#plot with standardized values

# Creating the scatter plot using go.Scatter
fig = go.Figure(data=go.Scatter(
    #x=data_Namen['time'],
    x = t,
    y=reconstructed_signal,
    mode='lines'
))

# Customizing the plot layout
fig.update_layout(
    title='Scatter Plot of LAEQ over Time',
    xaxis=dict(title='Time (hours)'),
    yaxis=dict(title='LAEQ')
)

# Displaying the plot
fig.show()